In [1]:
import sys
import numpy as np
from scipy.sparse import csr_matrix


In [64]:
def carrega_arquivo(nome_arquivo,tamanhoGrid):

    crs = open(nome_arquivo, "r")
    curenty_action = ''
    list_states = []
    dict_actions = {}
    initial_state = ''
    goal_state = ''
    
    session = [['states', 'endstates'], ['action', 'endaction'], ['cost', 'endcost'],
               ['initialstate', 'endinitialstate'], ['goalstate', 'endgoalstate']]
    curenty_session = ''

    index_to_state = {}
    cont = 1
    
    M = np.zeros(tamanhoGrid*tamanhoGrid+1)
    tamanhoGrid = tamanhoGrid + 1
    for i in range(1, tamanhoGrid):
        for j in range(1, tamanhoGrid):
            state = 'robot-at-x{a}y{b}'.format(a=j, b=i)
            index_to_state[state] = cont
            M[cont] = tamanhoGrid-j + tamanhoGrid-i - 2
            cont += 1
            

    matrix_curenty_index_state = np.array([])
    matrix_next_index_state = np.array([])
    matrix_curenty_index_action = np.array([])
    matrix_propabilities = np.array([])
    matrix_cost = np.array([])

    matrix_move_south = []
    matrix_move_north = []
    matrix_move_west = []
    matrix_move_east = []

    matrix_cost_move_south = []
    matrix_cost_move_north = []
    matrix_cost_move_west = []
    matrix_cost_move_east = []

    for columns in (raw.strip().split() for raw in crs):
        if len(columns) != 0:

            if columns[0] == 'states':
                curenty_session = session[0][0]

            elif columns[0] == session[0][1]:
                curent_action = ''

            elif columns[0] == 'action':
                curenty_session = session[1][0]
                curenty_action = columns[1]

            elif columns[0] == session[1][1]:
                if curenty_action == 'move-south':
                    matrix_move_south = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
                    matrix_cost_move_south = csr_matrix(
                        (matrix_cost, (matrix_curenty_index_state, matrix_next_index_state)), shape=(tamanhoGrid*tamanhoGrid, tamanhoGrid*tamanhoGrid))
                    matrix_curenty_index_state = []
                    matrix_next_index_state = []
                    matrix_curenty_index_action = []
                    matrix_propabilities = []
                    matrix_cost = []

                elif curenty_action == 'move-north':
                    matrix_move_north = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
                    matrix_cost_move_north = csr_matrix(
                        (matrix_cost, (matrix_curenty_index_state, matrix_next_index_state)),  shape=(tamanhoGrid*tamanhoGrid, tamanhoGrid*tamanhoGrid))
                    matrix_curenty_index_state = []
                    matrix_next_index_state = []
                    matrix_curenty_index_action = []
                    matrix_propabilities = []
                    matrix_cost = []

                elif curenty_action == 'move-west':
                    matrix_move_west = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
                    matrix_cost_move_west = csr_matrix(
                        (matrix_cost, (matrix_curenty_index_state, matrix_next_index_state)),  shape=(tamanhoGrid*tamanhoGrid, tamanhoGrid*tamanhoGrid))
                    matrix_curenty_index_state = []
                    matrix_next_index_state = []
                    matrix_curenty_index_action = []
                    matrix_propabilities = []
                    matrix_cost = []

                elif curenty_action == 'move-east':
                    matrix_move_east = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
                    matrix_cost_move_east = csr_matrix(
                        (matrix_cost, (matrix_curenty_index_state, matrix_next_index_state)),  shape=(tamanhoGrid*tamanhoGrid, tamanhoGrid*tamanhoGrid))
                    matrix_curenty_index_state = []
                    matrix_next_index_state = []
                    matrix_curenty_index_action = []
                    matrix_propabilities = []
                    matrix_cost = []

                curent_action = ''

            elif columns[0] == 'cost':
                curenty_session = session[2][0]

            elif columns[0] == session[2][1]:
                curent_action = ''

            elif columns[0] == 'initialstate':
                curenty_session = session[3][0]

            elif columns[0] == session[3][1]:
                curent_action = ''

            elif columns[0] == 'goalstate':
                curenty_session = session[4][0]

            elif columns[0] == session[4][1]:
                curent_action = ''
                break


            else:
                if len(columns) > 0:
                    if curenty_session == session[0][0]:  # states
                        list_states = []
                        for a in columns:
                            replaced_state = a.replace(',','')
                            list_states.append(index_to_state[replaced_state])

                    elif curenty_session == session[1][0]:  # action
                        
                        curenty_index_state = index_to_state[columns[0]]
                        next_index_state = index_to_state[columns[1]]
                        propabilities = columns[2]

                        matrix_curenty_index_state = np.append(matrix_curenty_index_state, int(curenty_index_state),
                                                               axis=None)
                        matrix_next_index_state = np.append(matrix_next_index_state, int(next_index_state), axis=None)
                        matrix_propabilities = np.append(matrix_propabilities, float(propabilities), axis=None)
                        matrix_cost = np.append(matrix_cost, 1, axis=None)

                    elif curenty_session == session[2][0]:  # cost
                        cost = 1

                    elif curenty_session == session[3][0]:  # initial_state
                        initial_state = columns[0]

                    elif curenty_session == session[4][0]:  # goal_state
                        goal_state = index_to_state[columns[0]]
                    
    #print(matrix_move_south)
    matrix_probabilties = [matrix_move_south, matrix_move_north, matrix_move_west, matrix_move_east]
    matrix_costs = [matrix_cost_move_south, matrix_cost_move_north, matrix_cost_move_west, matrix_cost_move_east]
    
    return matrix_probabilties, matrix_costs, curenty_action, list_states, dict_actions, initial_state, goal_state, M


In [103]:
def valueIteration(matrix_probabilties, list_states, goal_state, numStates, ehpistola, V):
    A = len(matrix_probabilties)
    S = numStates
   
    num_iteracoes = 0
    meta = goal_state
    V[meta] = 0
    while True:
        Q = np.zeros((A, S))
        for a in range(A):
            for s in list_states:

                if matrix_probabilties[a][s].getnnz() != 0:
                    if s != meta:
                        Q[a][s] = 1
                    for p in matrix_probabilties[a][s]:
                        data = p.data
                        indice = p.indices
                        for i in range(len(data)):
                            Q[a][s] += 0.99 * data[i] * V[indice[i]]
                    # print('a', a,'s', s,'data', data, 'indice', indice, 'Q', Q[a][s])
                else:
                    Q[a][s] = sys.maxsize - 1000

        num_iteracoes += 1
        print(num_iteracoes)

        v_antigo = V.copy()

        for s in range(S):
            minimo = sys.maxsize
            for a in range(A):
                if Q[a][s] < minimo:
                    minimo = Q[a][s]
            V[s] = minimo
        print(max(abs(v_antigo - V)))
        if max(abs(v_antigo - V)) < ehpistola:
            break

    return V

In [104]:
nome_arquivo = "C:/Users/Avell/Documents/2019 - 02/TPIA/EP2/TPIA2/TestesGrid/DeterministicGoalState/navigation_1.net"

tamGrid = 20

matrix_probabilities, matrix_costs, curenty_action, list_states, dict_actions, initial_state, goal_state, M = carrega_arquivo(
    nome_arquivo,tamGrid)

if goal_state == None:
    raise('Arquivo com formato diferente')
    
S = tamGrid*tamGrid+1
V = np.zeros(S) + sys.maxsize - 1000
politica = valueIteration(matrix_probabilities, list_states, goal_state, S , 0.01, V)

1
9.223372036854775e+18
2
2.3512681164952033e+18
3
2.305155867996948e+18
4
1.7395451965626962e+18
5
1.7055373676395428e+18
6
1.4280829851538954e+18
7
1.400233996422295e+18
8
1.2300791079471698e+18
9
1.2061424334141445e+18
10
1.0892533350469315e+18
11
1.0680969052609167e+18
12
9.82053276630313e+17
13
9.63011402708905e+17
14
8.966502805611581e+17
15
8.792915010370079e+17
16
8.26360455660074e+17
17
8.10385660092373e+17
18
7.67080216525779e+17
19
7.522715504010967e+17
20
7.161295693517732e+17
21
7.02322244214955e+17
22
6.716723871018721e+17
23
6.587379703851745e+17
24
6.324016437802322e+17
25
6.202375816159652e+17
26
5.97357382907881e+17
27
5.858801275244672e+17
28
5.658159310373724e+17
29
5.549562619460019e+17
30
5.372195619096668e+17
31
5.2691929987876045e+17
32
5.111302251115131e+17
33
5.013398624042532e+17
34
4.871981673079716e+17
35
4.778751192804434e+17
36
4.651400869790623e+17
37
4.562473762567767e+17
38
4.447235736652585e+17
39
4.362288224523443e+17
40
4.257557977375355e+17
41
4.17

In [66]:
print(M)

[ 0. 38. 37. 36. 35. 34. 33. 32. 31. 30. 29. 28. 27. 26. 25. 24. 23. 22.
 21. 20. 19. 37. 36. 35. 34. 33. 32. 31. 30. 29. 28. 27. 26. 25. 24. 23.
 22. 21. 20. 19. 18. 36. 35. 34. 33. 32. 31. 30. 29. 28. 27. 26. 25. 24.
 23. 22. 21. 20. 19. 18. 17. 35. 34. 33. 32. 31. 30. 29. 28. 27. 26. 25.
 24. 23. 22. 21. 20. 19. 18. 17. 16. 34. 33. 32. 31. 30. 29. 28. 27. 26.
 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 33. 32. 31. 30. 29. 28. 27.
 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14. 32. 31. 30. 29. 28.
 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14. 13. 31. 30. 29.
 28. 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14. 13. 12. 30.
 29. 28. 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14. 13. 12.
 11. 29. 28. 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14. 13.
 12. 11. 10. 28. 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15. 14.
 13. 12. 11. 10.  9. 27. 26. 25. 24. 23. 22. 21. 20. 19. 18. 17. 16. 15.
 14. 13. 12. 11. 10.  9.  8. 26. 25. 24. 23. 22. 21

In [105]:
def onde_esta_o_LAO(G, M, initial_state, goal_state, S):
    V = np.zeros(S) + sys.maxsize - 1000
    
    while(!converged):
        expanded = np.zeros(S)
        g1 = initial_state
        z = [1]
        n = 1
        expanded[1] = 1
        lista_sucessores = expandPartialSolution(G, g1, n)
        
        while(bestG.tipState == nonTerminal):
            n = melhorN(lista_sucessores, M, n)
            lista_sucessores = expandPartialSolution(n)
            expanded[n] = 1
            z.append(n)
            valueIteration(G, z, goal_state, S, V)
            
        converged = convergenceTest(g1, z, goal_state, S, V)

    return bestG

SyntaxError: invalid syntax (<ipython-input-105-89b781b6b3c1>, line 7)

In [ ]:
def convergenceTest(G, z, goal_state, tamGrid, V):
    valueIteration(G, z, goal_state, tamGrid, V)

In [99]:
def melhorN(lista_sucessores, M, n):
    lista_sucessores.remove(n)
    return lista_sucessores[np.argmin(M[lista_sucessores])]

In [97]:
def expandPartialSolution(g, g1, n):
    lista_sucessores = []
    A = 4
    for a in range(A):
        if g[a][n].getnnz() != 0:
            for p in g[a][n]:
                indice = p.indices
                for i in range(len(indice)):
                    if indice[i] not in lista_sucessores:
                        lista_sucessores.append(indice[i])
    
    for sucessor in lista_sucessores:
        for a in range(A):
            g1[a][sucessor] = g[a][sucessor]
    
    return lista_sucessores


In [100]:
matrix_propabilities = []
matrix_curenty_index_state = []
matrix_next_index_state = []
tamanhoGrid = 21
matrix_move_south = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
matrix_move_north = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
matrix_move_west = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))
matrix_move_east = csr_matrix(
                        (matrix_propabilities, (matrix_curenty_index_state, matrix_next_index_state)),
                        shape=(tamanhoGrid * tamanhoGrid, tamanhoGrid * tamanhoGrid))

g1 = [matrix_move_south, matrix_move_north, matrix_move_west, matrix_move_east]
n = 1
lista_sucessores = expandPartialSolution(matrix_probabilities, g1, n)
melhorN(lista_sucessores, M, n)


21

In [ ]:
def convergenceTest: